In [1]:
from pathlib import Path
from zipfile import ZipFile, BadZipFile
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import boto3

In [2]:
bucket_url = "biorxiv-src-monthly"

In [3]:
%%time
client = boto3.client("s3")

result = client.list_objects_v2(Bucket=bucket_url, RequestPayer="requester")
key_list = [r["Key"] for r in result["Contents"]]

while result["IsTruncated"]:
    result = client.list_objects_v2(
        Bucket=bucket_url,
        RequestPayer="requester",
        ContinuationToken=result["NextContinuationToken"],
    )
    key_list += [r["Key"] for r in result["Contents"]]

print(len(key_list))

111154
CPU times: user 17.5 s, sys: 396 ms, total: 17.9 s
Wall time: 30.5 s


In [4]:
def get_xml(key):
    p = Path(key)
    if p.suffix == ".meca":
        target_folder = p.with_suffix("")
        if not target_folder.exists():
            target_folder.mkdir(exist_ok=True, parents=True)
            client = boto3.client("s3")
            client.download_file(
                bucket_url, key, key, ExtraArgs={"RequestPayer": "requester"}
            )
            try:
                with ZipFile(key, "r") as zipObj:
                    for sub_file in zipObj.namelist():
                        if sub_file.endswith(".xml"):
                            zipObj.extract(sub_file, target_folder)
            except BadZipFile:
                pass
            p.unlink()

In [5]:
%%time
PARALLEL = True
if PARALLEL:
    Parallel(n_jobs=32)(delayed(get_xml)(key) for key in tqdm(key_list))
else:
    [get_xml(key) for key in tqdm(key_list)]


CPU times: user 1min 5s, sys: 5.05 s, total: 1min 10s
Wall time: 19min 43s
